In [2]:
import polars as pl

from config import (COLS_TO_SELECT, COLS_ORDERED, LOCATION_COL, SELLER_PROFESSIONAL, GEARBOX_ID, AD_TYPE, CAR_BODY, COLS_TRANSFORMED, COLS_RENAME_DICT, CAR_BODY_DICT)

In [3]:
pl.Config.set_fmt_str_lengths(40)
pl.Config.set_tbl_rows(20)

polars.config.Config

In [4]:
df = pl.read_parquet('../02_load/car_ads_load.parquet')

In [5]:
df.shape

(185418, 21)

In [6]:
# There are 37 cars for which the ad price was adjusted AND the MILEAGE was different. 
# This raise alarms about potentially trying to artificially adjust mileage:
(
    df
    .with_columns(pl.col('id','seller_name','title').hash())
    .filter(pl.col('id').is_duplicated())
    .sort(by='id')
    .group_by('seller_name','id','title')
    .agg(pl.len(),
        pl.col('price_amount').min().alias('min_price'),
         pl.col('price_amount').max().alias('max_price'),
         pl.col('km').min().alias('min_km'),
         pl.col('km').max().alias('max_km'),
         ((pl.col('km').max().alias('max_km') - pl.col('km').min().alias('min_km')).alias('km_diff')),
         (pl.col('price_amount').max().alias('max_price') - pl.col('price_amount').min().alias('min_price')).alias('price_diff'))
    .sort(by='seller_name', descending=True)
    .filter(pl.col('km_diff') > 0)
).head(5)

seller_name,id,title,len,min_price,max_price,min_km,max_km,km_diff,price_diff
u64,u64,u64,u32,i64,i64,i64,i64,i64,i64
18061391965307345239,12568132951389192755,16348551259640286522,2,35790,35990,19632,52219,32587,200
17832316939368330321,14988602387834855962,14529253179343365493,2,14500,14700,137000,139000,2000,200
17696703690871970547,10806452780687109007,15099564523844778274,2,18500,19000,168000,169000,1000,500
17696703690871970547,12894711516793685280,13167333921448933609,2,18500,19000,168000,169000,1000,500
17350074703501238312,4772691319298227880,13846742274461279224,2,11990,12000,240000,242000,2000,10


In [7]:
df = (
    df
    .select(
        COLS_TO_SELECT
    )
)

In [8]:
# How many cars don't have modelId or km:
(
    df
    .filter(
        (pl.col('modelId').is_null()) |
        (pl.col('km').is_null()) |
        (pl.col('year').is_null())
    )
).__len__()

183

In [9]:
# We filter out cars without modelId, km or year as they represent merely 0.1% of the whole dataset.
df = (
    df
    .filter(
        (~pl.col('modelId').is_null()),
        (~pl.col('km').is_null()),
        (~pl.col('year').is_null())
    )
)

In [10]:
# How many cars don't have fuelType:
sum(df['fuelType'].is_null())

910

In [11]:
# We can infer the fuel type for the missing values from the url:
sum(df.filter((pl.col('fuelType').is_null()))['url'].str.contains('hibrido'))

734

In [12]:
sum(df.filter((pl.col('fuelType').is_null()))['url'].str.contains('otros'))

176

In [13]:
# We update the missing fuelType values. 
# In the case of 'otros'(other) we'll use Gasoline as is the 2nd most fuelType
# and observing the models is a better choice than Diesel.
# As well we adjust one value from Diesel to Diésel. 
df = (
    df
    .with_columns(
        pl.when(pl.col('fuelType').is_null() & pl.col('url').str.contains('hibrido'))
        .then(pl.lit('Híbrido'))
        .when(pl.col('fuelType').is_null() & pl.col('url').str.contains('otros'))
        .then(pl.lit('Gasolina'))
        .when(pl.col('fuelType') == 'Diesel')
        .then(pl.lit('Diésel'))
        .otherwise(pl.col('fuelType'))
        .alias('fuelType')
    )
)

In [14]:
df['fuelType'].value_counts(sort=True)

fuelType,count
str,u32
"""Diésel""",90266
"""Gasolina""",68510
"""Híbrido""",15705
"""Híbrido enchufable""",6054
"""Eléctrico""",3863
"""Gas licuado (GLP)""",664
"""Gas natural (CNG)""",173


In [15]:
# As there are only 12 rows with missing location province we update it to 'Madrid' as this is by far the most common province:
df = (
    df
    .with_columns(
        pl.when(pl.col(LOCATION_COL).is_null())
            .then(pl.lit("Madrid"))
            .otherwise(pl.col(LOCATION_COL))
            .alias(LOCATION_COL)
    )
)

In [16]:
df[SELLER_PROFESSIONAL].value_counts(sort=True, normalize=True)

seller_isProfessional,proportion
bool,f64
true,0.614009
false,0.385991


In [17]:
df[AD_TYPE].value_counts(sort=True, normalize=True).head(2)

offerType_literal,proportion
str,f64
"""Ocasión""",0.980814
"""Km0""",0.015942


In [18]:
# The missing values of the bodyTypeId column can't be clearly inferred by grouping the modelId
#  and taking the median so we'll ignore it for the moment:
(
    df
    .group_by('modelId')
    .agg(pl.col(CAR_BODY).median())
)[CAR_BODY].value_counts(sort=True).shape

(12, 2)

In [19]:
# Convert the 'creationDate' column to a datetime type
df = df.with_columns(
    pl.col("creationDate").str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%SZ"),
    pl.col("publishedDate").str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%SZ"))


In [20]:
# # Define the specific date as a datetime object
specific_date = pl.date(2024, 9, 4)

# # Calculate the difference in days
df = df.with_columns(
    (specific_date - pl.col("creationDate").dt.date()).dt.total_days().alias('ad_days_creation'),
    (specific_date - pl.col("publishedDate").dt.date()).dt.total_days().alias('ad_days_published'))

In [21]:
# We'll later choose ad_days_creation as in case it differs from published, creation date is earlier (in most cases).

(
    df
    .filter(
        pl.col('ad_days_creation') != pl.col('ad_days_published'),
        pl.col('creationDate') < pl.col('publishedDate')
    )
).shape[0]

58963

In [22]:
brand_models_df = pl.read_csv('../02_load/brand_models.csv')

In [23]:
df = (df
    .join(brand_models_df,
            left_on= ['makeId', 'modelId'],
            right_on=['brand_id', 'model_id'],
            how='left')
    )

In [24]:
df = (
    df.with_columns(
        pl.col('brand','model').str.to_lowercase()
    )
)

In [25]:
# There are some brands that still have models without name:
(
    df
    .filter(pl.col('brand').is_null())
)['makeId'].value_counts(sort=True).shape

(2, 2)

In [26]:
# There are only 2 items without brand. Those ads are not relevant (one is not even about a car)
(df
    .filter(pl.col('model').is_null())
    ).unique(subset='modelId')[['makeId','modelId','title']].sort(by='title').shape

(2, 3)

In [27]:
# We filter out the few items (2 items) without brand as they are not relevant car ads:
df = (
    df
    .filter(~pl.col('brand').is_null())
)

In [28]:
df = (
    df
    .select(
        COLS_ORDERED
    )
)

In [29]:
# We create an 'age_years' column so that we can interpret how car prices changes based on how old is it:
df = (
    df
    .with_columns(
        (pl.lit(2024) - pl.col('year')).alias('age_years')
    )
)

In [30]:
# df.describe()

In [31]:
df_transform = (
    df
    .select(
        COLS_TRANSFORMED
    )
)

In [32]:
df_transform = (
    df_transform
    .with_columns(
        pl.col(pl.Int64).cast(pl.Int32),
        pl.col(pl.Utf8).str.to_lowercase().str.replace_many(['á','é',"í","ó","ú"], ['a','e','i','o','u'])
    ))

In [33]:
df_transform = (
    df_transform
    .with_columns(pl.col(GEARBOX_ID) == 1)
    .rename(COLS_RENAME_DICT)
)

## Car Body Type preprocessing:

In [34]:
brand_model_bodyType_map = df.group_by(['brand','model']).agg(pl.col(CAR_BODY).median().cast(int))

In [35]:
df_transform = (df_transform
    .join(brand_model_bodyType_map,
            left_on= ['brand','model'],
            right_on=['brand','model'],
            how='left')
    .with_columns(
        pl.col(CAR_BODY).fill_null(pl.col('bodyTypeId_right'))
    )
    .drop('bodyTypeId_right')
)

In [36]:
df_transform = df_transform.with_columns(pl.col(CAR_BODY).cast(pl.Int32))

In [37]:
body_type_dict = CAR_BODY_DICT

body_type_dict_df = pl.DataFrame(body_type_dict, schema={CAR_BODY:pl.Int32, 'body_type':pl.Utf8})

In [38]:
df_transform = (df_transform
    .join(body_type_dict_df,
            left_on= [CAR_BODY],
            right_on=[CAR_BODY],
            how='left')
    )

In [40]:
# Number of unique values per column. Useful to then do analysis and create dummy variables for modelling:
df_transform.select(df_transform.columns[:-5]).select(pl.all().n_unique())

price,brand,model,is_automatic,km,fuel,year,age_years,is_seller_pro,is_certified,offer_type,location,ad_days_creation,ad_days_published
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
5674,96,1182,2,46808,7,55,55,2,2,5,52,75,76


In [41]:
# The df is ready to be explored and analysed to understand the distributions and categories, to gain insights and prepare the car price modeling:
df_transform.write_parquet('car_ads_transformed.parquet')

In [42]:
df_transform = pl.read_parquet('car_ads_transformed.parquet')

In [43]:
df_transform.shape

(185233, 19)

In [45]:
df_transform.select(df_transform.columns[:-5]).head(2)

price,brand,model,is_automatic,km,fuel,year,age_years,is_seller_pro,is_certified,offer_type,location,ad_days_creation,ad_days_published
i32,str,str,bool,i32,str,i32,i32,bool,bool,str,str,i32,i32
12990,"""mini""","""mini""",true,165399,"""diesel""",2015,9,true,false,"""ocasion""","""madrid""",0,0
10500,"""audi""","""a1""",false,143139,"""diesel""",2013,11,true,false,"""ocasion""","""madrid""",0,0
